## **Problem Statement**
Saudi Aramco is one of the largest publicly traded companies, and its stock price is influenced by various factors such as market trends, oil prices, economic policies, and investor sentiment. Predicting stock prices accurately is crucial for investors, financial analysts, and stakeholders to make informed decisions. However, stock prices are highly volatile, making accurate prediction a complex challenge.

This project aims to develop a machine learning model to forecast Saudi Aramco’s stock closing prices based on historical data and various technical indicators. The goal is to identify meaningful patterns and trends that can help predict future stock prices, improving investment strategies.

## **Project Goals & Objectives**
**1 - Analyze Stock Price Trends**
- Examine historical Saudi Aramco stock data to identify key patterns.
- Study the impact of trading volume, market trends, and technical indicators on stock prices.

**2 - Build a Predictive Regression**
- Use AI and statistical regression techniques to forecast future stock prices.
- Train the model on historical data to improve prediction accuracy.

**3 - Support Data-Driven Investment Decisions**
- Provide investors and analysts with insights to make informed decisions.
- Reduce uncertainty by predicting potential stock price movements.

**4 - Ensure Model Accuracy & Reliability**
- Validate the model using real-world data and key performance metrics.
- Measure accuracy using RMSE, MAE, and R² scores.

**5 - Make Predictions Accessible & Understandable**
- Visualize predictions using charts, graphs, and trend reports.
- Optionally, develop a simple dashboard for interactive forecasting.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pickle as pkl
import warnings
warnings.filterwarnings('ignore')

#### Loading the dataset

In [4]:
df = pd.read_csv('aramco_stock_price_dataset.csv', encoding='utf-8')

#### Understand the data 

In [5]:
df.head(3)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Close_diff,Lag_Close,...,BB_Middle_Band,BB_Upper_Band,BB_Lower_Band,Change_Close,Change_Volume,Weekday,Month,Year,Quarter,Volume_Normalized
0,2019-12-11 00:00:00+03:00,25.485229,25.485229,25.485229,25.485229,38289394,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2,12,2019,4,1.580604
1,2019-12-12 00:00:00+03:00,28.019275,28.019275,26.064442,26.643652,505692621,0.0,0.0,1.158422,25.485229,...,NaN,NaN,NaN,1.158422,467403227.0,3,12,2019,4,27.146985
2,2019-12-15 00:00:00+03:00,26.860858,27.150462,26.643654,27.078060,98349281,0.0,0.0,0.434408,26.643652,...,NaN,NaN,NaN,0.434408,-407343340.0,6,12,2019,4,4.865806


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               1095 non-null   object 
 1   Open               1095 non-null   float64
 2   High               1095 non-null   float64
 3   Low                1095 non-null   float64
 4   Close              1095 non-null   float64
 5   Volume             1095 non-null   int64  
 6   Dividends          1095 non-null   float64
 7   Stock Splits       1095 non-null   float64
 8   Close_diff         1094 non-null   float64
 9   Lag_Close          1094 non-null   float64
 10  Lag_High           1094 non-null   float64
 11  Lag_Low            1094 non-null   float64
 12  Rolling_Mean_7     1089 non-null   float64
 13  Rolling_Std_7      1089 non-null   float64
 14  Rolling_Mean_30    1066 non-null   float64
 15  Rolling_Std_30     1066 non-null   float64
 16  RSI                1082 

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Open,1095.0,2.885776e+01,3.472621e+00,1.954834e+01,2.654623e+01,2.843185e+01,3.184231e+01,3.641110e+01
High,1095.0,2.903354e+01,3.503517e+00,2.048955e+01,2.665929e+01,2.856368e+01,3.203948e+01,3.713932e+01
Low,1095.0,2.867893e+01,3.421617e+00,1.954834e+01,2.643623e+01,2.821213e+01,3.169444e+01,3.615408e+01
Close,1095.0,2.886365e+01,3.458323e+00,2.012754e+01,2.654817e+01,2.847579e+01,3.184231e+01,3.641110e+01
Volume,1095.0,9.392873e+06,1.829030e+07,0.000000e+00,3.777274e+06,6.202220e+06,1.108638e+07,5.056926e+08
Dividends,1095.0,3.695110e-03,3.158860e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.024000e-01
Stock Splits,1095.0,4.018265e-03,6.639255e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.100000e+00
Close_diff,1094.0,4.081144e-03,3.196540e-01,-2.172035e+00,-1.134429e-01,0.000000e+00,1.157408e-01,2.027229e+00
Lag_Close,1094.0,2.886265e+01,3.459748e+00,2.012754e+01,2.654735e+01,2.847579e+01,3.184231e+01,3.641110e+01
Lag_High,1094.0,2.903257e+01,3.504971e+00,2.048955e+01,2.665837e+01,2.855394e+01,3.203948e+01,3.713932e+01


In [9]:
df.isnull().sum()

Date                  0
Open                  0
High                  0
Low                   0
Close                 0
Volume                0
Dividends             0
Stock Splits          0
Close_diff            1
Lag_Close             1
Lag_High              1
Lag_Low               1
Rolling_Mean_7        6
Rolling_Std_7         6
Rolling_Mean_30      29
Rolling_Std_30       29
RSI                  13
MACD                  0
BB_Middle_Band       19
BB_Upper_Band        19
BB_Lower_Band        19
Change_Close          1
Change_Volume         1
Weekday               0
Month                 0
Year                  0
Quarter               0
Volume_Normalized     0
dtype: int64

In [10]:
df.shape

(1095, 28)

In [11]:
df.dtypes

Date                  object
Open                 float64
High                 float64
Low                  float64
Close                float64
Volume                 int64
Dividends            float64
Stock Splits         float64
Close_diff           float64
Lag_Close            float64
Lag_High             float64
Lag_Low              float64
Rolling_Mean_7       float64
Rolling_Std_7        float64
Rolling_Mean_30      float64
Rolling_Std_30       float64
RSI                  float64
MACD                 float64
BB_Middle_Band       float64
BB_Upper_Band        float64
BB_Lower_Band        float64
Change_Close         float64
Change_Volume        float64
Weekday                int64
Month                  int64
Year                   int64
Quarter                int64
Volume_Normalized    float64
dtype: object